Noisy circuit analysis

Import libraries and modules

In [1]:
import sys
sys.path.append('../src')

import numpy as np
import ipympl

from noisy_circuits_Hamming import hamming_circuits
from noisy_circuits_Hellinger import hellinger_circuits

from utils import simulate_with_noise_3D
from utils import DistanceVisualizer
from utils import plot_3d

print("All imports successful!")

All imports successful!


Workflow:
1. Simulate circuit with Hamming output 10 times over ranges of shots and noise
2. Save to file
3. Load from file
4. Visualize Hamming distance of run of choice with respect to number of shots. STD of all hamming distances
   is displayed on top and noise can be regulated.

Experiment 1:
Repeating CNOT

In [ ]:
circuit_index = 5

circuits, true_outputs = hamming_circuits()
circuit, true_output = circuits[circuit_index], true_outputs[circuit_index]

noise_levels = np.linspace(0, 0.3, 51)
shot_counts = np.arange(10, 1001, 10)

# Uncomment this when you want to generate data for a specific circuit

# # Simulate and collect Hamming distance data 10 times for averaging
# data = []
# for i in range(10):
#     print(f"Simulation run {i+1}/10 for circuit index {circuit_index}")
#     H = simulate_with_noise_3D(
#         circuit, true_output, noise_levels=noise_levels, shot_counts=shot_counts, distance_type='hamming'
#     )
#     data.append(H)

# # Save the data
# np.savez_compressed(
#     f'../data/circuit_{circuit_index}_hamming_data.npz',
#     shots_array=shot_counts,
#     noise_intensities=noise_levels,
#     hamming_data=data
# )

In [ ]:
# Load the data
loaded = np.load(f'../data/circuit_{circuit_index}_hamming_data.npz', allow_pickle=True)
shot_counts = loaded['shots_array']
noise_levels = loaded['noise_intensities']
hamming_data = loaded['hamming_data']

visualizer = DistanceVisualizer(shot_counts, noise_levels, hamming_data, circuit.num_qubits)
visualizer.create_dashboard()

Experiment 3:
QFT followed by inverse QFT on |101>

In [2]:
circuit_index = 1

hamming_circuits, hamming_true_outputs = hamming_circuits()
hellinger_circuits, hellinger_ideal_dists = hellinger_circuits()

mirrored_qft_circuit, true_output = hamming_circuits[circuit_index], hamming_true_outputs[circuit_index]
qft_circuit, ideal_dist = hellinger_circuits[circuit_index], hellinger_ideal_dists[circuit_index]

noise_levels = np.linspace(0, 0.2, 21)

# Create ranges for each segment
ranges = [
    np.arange(10, 250, 10),      # Usually big std reduction occurs here so we want to most precision
    np.arange(250, 1000, 50),   # Here it slowly starts to converge
    np.arange(1000, 2000, 100), # Usually completely converges in this range
    np.arange(2000, 10001, 500)
]
shot_counts = np.concatenate(ranges)

# Uncomment this when you want to generate data for a specific circuit

# Simulate and collect Hellinger distance on 1 run
hellinger_data = simulate_with_noise_3D(
         mirrored_qft_circuit, true_output, noise_levels=noise_levels, shot_counts=shot_counts, distance_type='hamming'
)

# # Simulate and collect Hamming distance data 10 runs for averaging
# hamming_data = []
# for i in range(10):
#     print(f"Simulation run {i+1}/10 for circuit index {circuit_index}")
#     H = simulate_with_noise_3D(
#         mirrored_qft_circuit, true_output, noise_levels=noise_levels, shot_counts=shot_counts, distance_type='hamming'
#     )
#     hamming_data.append(H)

# # Save the data
# np.savez_compressed(
#     f'../data/circuit_{circuit_index}_hamming_data.npz',
#     shots_array=shot_counts,
#     noise_intensities=noise_levels,
#     hamming_data=hamming_data
# )

     ┌──────────────────────────────┐┌──────┐┌─┐      
q_0: ┤0                             ├┤0     ├┤M├──────
     │                              ││      │└╥┘┌─┐   
q_1: ┤1 Initialize(0,1,0,0,0,0,0,0) ├┤1 Qft ├─╫─┤M├───
     │                              ││      │ ║ └╥┘┌─┐
q_2: ┤2                             ├┤2     ├─╫──╫─┤M├
     └──────────────────────────────┘└──────┘ ║  ║ └╥┘
c: 3/═════════════════════════════════════════╩══╩══╩═
                                              0  1  2 
Simulating circuit with noise range [0.0, 0.2] and shot counts [10, 10000]
Using distance metric: hamming


In [ ]:
# Load the data
loaded = np.load(f'../data/circuit_6_hamming_data.npz', allow_pickle=True)
shot_counts = loaded['shots_array']
noise_levels = loaded['noise_intensities']
hamming_data = loaded['hamming_data']

visualizer = DistanceVisualizer(shot_counts, noise_levels, hamming_data, hellinger_data, mirrored_qft_circuit.num_qubits)
visualizer.create_dashboard()

interactive(children=(FloatSlider(value=0.1, description='Noise Intensity:', max=0.2, step=0.00952380952380952…

Experiment 4 a variational circuit